In [1]:
import pandas as pd
import numpy as np
from DataReader import split_sheet_into_test_training_per_word
from nltk.tokenize import regexp_tokenize
from nltk import bigrams
from nltk import FreqDist
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
x_test, x_train, y_test, y_train = split_sheet_into_test_training_per_word()

In [2]:
def process_1(x, y, n):
    start_symbol = '<s>'
    clean = []
    for index, row in x.iterrows():
        if row['probe'] in row['line']:
            line = regexp_tokenize(row['line'], "[\w']+")
            label = row['probe']+str(y.loc[index][0])
            ind= line.index(row['probe'])
            if ind-n+1>0:
                s= line[ind-n+1:ind]
                s.append(label)
            elif ind!=0:
                s = [start_symbol]*(n-1-ind)
                s.extend(line[0:ind])
                s.append(label)
            else:
                s = [start_symbol]*(n-1)
                s.append(label)
            clean.append(s)
    return clean

In [3]:
n=2
start_symbol = '<s>'
x_train_clean = [] #process_1(x_train, y_train, n)
for index, row in x_train.iterrows():
    if row['probe'] in row['line']:
        line = regexp_tokenize(row['line'], "[\w']+")
        label = row['probe']+str(y_train.loc[index][0])
        ind= line.index(row['probe'])
        if ind-n+1>0:
            s= line[ind-n+1:ind]
            s.append(label)
        elif ind!=0:
            s = [start_symbol]*(n-1-ind)
            s.extend(line[0:ind])
            s.append(label)
        else:
            s = [start_symbol]*(n-1)
            s.append(label)
        x_train_clean.append(s)

TypeError: expected string or bytes-like object

In [ ]:
train_data, padded_sents = padded_everygram_pipeline(n, x_train_clean)

model= MLE(n)
model.fit(train_data, padded_sents)

In [ ]:
correct, error, noidea=0,0,0

for index, row in x_test.iterrows():
    p=[]
    for i in range(4):
        line= regexp_tokenize(row['line'], "[\w']+")
        ind = line.index(row['probe'])
        prior =[]
        if ind-n+1>0:
            s= line[ind-n+1:ind]
            prior.extend(s)
        elif ind!=0:
            s = [start_symbol]*(n-1-ind)
            s.extend(line[0:ind])
            prior.extend(s)
        else:
            s = [start_symbol]*(n-1)
            prior.extend(s)
        p.append(model.score(row['probe']+str(i+1),prior))
    if sum(p)<0.00001:
        noidea+=1
    elif p.index(max(p))+1 == y_test.loc[index][0]:
        correct+=1
    else:
        error+=1

In [ ]:
print(correct)
print(error)
print(noidea)

In [7]:
def process_4(x, y, n):
    start_symbol = '<s>'
    clean1, clean2, clean3, clean4 = [], [], [], []
    for index, row in x.iterrows():
        if row['probe'] in row['line']:
            line = regexp_tokenize(row['line'], "[\w']+")
            label = row['probe']
            ind= line.index(row['probe'])
            if ind-n+1>0:
                s= line[ind-n+1:ind]
            elif ind!=0:
                s = [start_symbol]*(n-1-ind)
                s.extend(line[0:ind])
            else:
                s = [start_symbol]*(n-1)
            s.append(label)
            if y.loc[index][0]== 1:
                clean1.append(s)
            elif y.loc[index][0]== 2:
                clean2.append(s)
            elif y.loc[index][0]== 3:
                clean3.append(s)
            else:
                clean4.append(s)
    return clean1, clean2, clean3, clean4

In [8]:
clean1, clean2, clean3, clean4 = process_4(x_train, y_train, n)
train_data1, padded_sents1 = padded_everygram_pipeline(n, clean1)
train_data2, padded_sents2 = padded_everygram_pipeline(n, clean2)
train_data3, padded_sents3 = padded_everygram_pipeline(n, clean3)
train_data4, padded_sents4 = padded_everygram_pipeline(n, clean4)

model1= MLE(n)
model1.fit(train_data1, padded_sents1)
model2= MLE(n)
model2.fit(train_data2, padded_sents2)
model3= MLE(n)
model3.fit(train_data3, padded_sents3)
model4= MLE(n)
model4.fit(train_data4, padded_sents4)
models = [model1, model2, model3, model4]

In [10]:
correct, error, noidea=0,0,0

for index, row in x_test.iterrows():
    p=[]
    line= regexp_tokenize(row['line'], "[\w']+")
    ind = line.index(row['probe'])
    prior =[]
    if ind-n+1>0:
        s= line[ind-n+1:ind]
    elif ind!=0:
        s = [start_symbol]*(n-1-ind)
        s.extend(line[0:ind])
    else:
        s = [start_symbol]*(n-1)
    prior.extend(s)
    for i in range(len(models)):
        p.append(models[i].score(row['probe'],prior))
    if sum(p)<0.00001:
        noidea+=1
    elif p.index(max(p))+1 == y_test.loc[index][0]:
        correct+=1
    else:
        error+=1

In [11]:
print(correct)
print(error)
print(noidea)

7181
460
3744


In [6]:
' '.join(x_train.iloc[0]['line'])

'Males of the pack sense the progression of the mating dance as well . Fucked her ? Fucked her . Cool . The pack rejoices . Yes ! Over time , the male and female ... continue to practice the physicality of mating . Oh ! Yes !'